In [1]:
import mysql.connector as mysql
import os

In [3]:
db = mysql.connect(
    host = "localhost",
    user = "root",
    passwd = "root",
    database = "Basic_Info"
)
cursor = db.cursor()

In [18]:
cursor.execute("CREATE TABLE test(customer_id VARCHAR(255) PRIMARY KEY,customer_name VARCHAR(255),customer_email VARCHAR(255), customer_address VARCHAR(255), customer_number INT(10), feedback TEXT(1200) NOT NULL, time_of_feedback VARCHAR(255) NOT NULL, loaction VARCHAR(255), stopwords INT(9), punctuations INT(9), tokens INT(9),html_tag INT(9),hash_tags INT(9),special_characters INT(9),digits INT(9),extra_space INT(9))")

In [4]:
cursor.execute("SHOW TABLES")
tables = cursor.fetchall() ## it returns list of tables present in the database
## showing all the tables one by one
for table in tables:
    print(table)

('information',)
('test',)


In [5]:
cursor.execute("DESC test")

## it will print all the columns as 'tuples' in a list
print(cursor.fetchall())

[('customer_id', 'varchar(255)', 'NO', 'PRI', None, ''), ('customer_name', 'varchar(255)', 'YES', '', None, ''), ('customer_email', 'varchar(255)', 'YES', '', None, ''), ('customer_address', 'varchar(255)', 'YES', '', None, ''), ('customer_number', 'int(10)', 'YES', '', None, ''), ('feedback', 'text', 'NO', '', None, ''), ('time_of_feedback', 'varchar(255)', 'NO', '', None, ''), ('loaction', 'varchar(255)', 'YES', '', None, ''), ('stopwords', 'int(9)', 'YES', '', None, ''), ('punctuations', 'int(9)', 'YES', '', None, ''), ('tokens', 'int(9)', 'YES', '', None, ''), ('html_tag', 'int(9)', 'YES', '', None, ''), ('hash_tags', 'int(9)', 'YES', '', None, ''), ('special_characters', 'int(9)', 'YES', '', None, ''), ('digits', 'int(9)', 'YES', '', None, ''), ('emojis', 'int(9)', 'YES', '', None, ''), ('extra_space', 'int(9)', 'YES', '', None, '')]


In [9]:
cursor.execute("INSERT into test VALUES ('ci_101','test','st15840@gmail.com','gurugram,haryana','1201235690','Coronaviruses : are a group of54 viruses   that cause @ diseases in mammals and   birds.','2pm','gurugram',0 ,0 ,0 ,0 ,0 ,0 ,0 ,0 )")

In [44]:
cursor.execute("select* from test")
data = cursor.fetchone()
print(data)

('ci_101', 'test', 'st15840@gmail.com', 'gurugram,haryana', 1201235690, 'Coronaviruses are a group of viruses that cause diseases in mammals and birds.', '2pm', 'gurugram', 0, 0, 0, 0, 0, 0, 0, 0, 0)


In [45]:
cursor.execute("SELECT feedback FROM test")
result_set = cursor.fetchall()
text = []
for row in result_set:
    print(row)
    print(type(row))
    text.append(list(row))

('Coronaviruses are a group of viruses that cause diseases in mammals and birds.',)
<class 'tuple'>


In [46]:
new = ""
for i in text:
    for j in i:
        new = new + j

In [47]:
new

'Coronaviruses are a group of viruses that cause diseases in mammals and birds.'

In [115]:
import numpy as np
import re
from nltk import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
import string
import emoji
from math import pi
from itertools import islice
from sklearn.feature_extraction.text import CountVectorizer

In [116]:
HtmlTag = 'html_tag'
HtmltagPattern = '<.*?>'
HashtagPattern = '#(\w+)'
ExtraSpacesPattern = '  +'
EmojiPattern = '(:[^:]*:)'
ListPunctuation = ['.',',','?','!',';',"'",'"',':','[',']','(',')','-','—','..','``',"''",'--','...','…','....']
WordPattern = '^[a-zA-Z0-9_]+$'
#---------------------------------------------------------
# 1. Calculate count of Tokens
#--------------------------------------------------------
def tokenize(text):
    text = text.lower()
    text = re.sub("'s","",text)
    text = re.sub("'re","",text)
    text = re.sub("n't","",text)
    text = re.sub("'ll","",text)
    text = re.sub("'m","",text)
    text = re.sub("'ve","",text)
    text = re.sub("'d","",text)
    text = re.sub("(//[a-zA-Z_0-9]\.co/[a-zA-Z_0-9]*)","",text)
    tokenizer = word_tokenize(text)
    return tokenizer

def count_tokens(text):
    tokens = tokenize(text)
    dictTokens = dict(Counter(tokens))
    return len(dictTokens)

#---------------------------------------------------------
# 2. Calculate count of Punctuations
#---------------------------------------------------------

def count_punctuations(text):
    punc = [i for i in text if i in ListPunctuation]
    dictPunc = dict(Counter(punc))
    return len(dictPunc)

#---------------------------------------------------------
# 3. Calculate count of Stopwords
#---------------------------------------------------------
def count_stopwords(text):
    tokens = tokenize(text)
    stop_words = set(stopwords.words("english"))
    dictStopwords=[s for s in tokens if s in stop_words]
    dictStopwords = dict(Counter(dictStopwords))
    return len(dictStopwords)

#---------------------------------------------------------
# 4. Calculate count of Hashtags
#---------------------------------------------------------

def find_replace_html_tags(text):
    return re.sub(HtmltagPattern,' html_tag ', text)

def find_hastags(text):
    return re.findall(HashtagPattern, text)

def count_hashtags(text):
    text = find_replace_html_tags(text)
    listHashtags = []
    listHash = find_hastags(text)
    if len(listHash) > 0:
        listHashtags.append(listHash)
    return len(listHashtags)

#---------------------------------------------------------
# 5. Calculate count of Numeric Values
#---------------------------------------------------------
def count_digits(text):
    tokens = tokenize(text)
    listDigits = sum(i.isdigit() for i in tokens)
    return listDigits

#---------------------------------------------------------
# 6. Calculate count of Html Tags
#---------------------------------------------------------
def count_html_tag(text):
    tokens = tokenize(text)
    countHtmlTags = 0
    for i in tokens:
        if i == HtmlTag:
            countHtmlTags = countHtmlTags+1
    return countHtmlTags

# #---------------------------------------------------------
# # 7. Calculate count of Special Characters
# #---------------------------------------------------------
def count_special_characters(text):
    tokens = tokenize(text)
    listSpecialCharacter = []
    for i in tokens:
        i = re.sub("([a-zA-Z_0-9]\-*)","",i)
        i = re.sub("([a-zA-Z_0-9]\.*)","",i)
        i = re.sub('(\:[a-zA-Z_]*)','.',i)
        if i.isalpha()!=True:
            if i.isdigit()!=True:
                if i not in ListPunctuation:
                    if i != 'html_tag':
                        listSpecialCharacter.append(i)
    dictSpecialCharacter = dict(Counter(listSpecialCharacter))
    del dictSpecialCharacter['']
    return len(dictSpecialCharacter)

#---------------------------------------------------------
# 8. Calculate count of Extra Spaces
#---------------------------------------------------------

def count_extra_spaces(text):
    listExtraSpaces = []
    i = re.findall(ExtraSpacesPattern,text)
    if len(i)>0:
        listExtraSpaces.append(i)
    return len(listExtraSpaces)


In [118]:
tokens = count_tokens(new)
punc = count_punctuations(new)
stopwords = count_stopwords(new)
hashtag = count_hashtags(new)
digit = count_digits(new)
html = count_html_tag(new)
special_char = count_special_characters(new)
extraspaces = count_extra_spaces(new)

In [ ]:
# Export database file / store the file
os.system('mysqldump -u root -p%s Basic_Info > Basic_Info.sql' % 'root')